In [4]:
import pandas as pd
import numpy as np
import sqlite3

from bkcharts import Bar, cat
from bokeh.io import output_notebook, save
from bokeh.models import HoverTool
from bokeh.palettes import viridis
from bokeh.plotting import show
from scipy import stats

# import CleanData as cd
# import BokehPlots as bp
# import CoinagePlots as cp

import sys
stdout = sys.stdout
reload(sys)
sys.setdefaultencoding('utf-8')
sys.stdout = stdout

In [5]:
title = 'Roman_Imperial_Coinage'
fname = '../Data/' + title + '.sqlite'
conn = sqlite3.connect(fname)

# Functions

## Get Counts and Ratios Data

In [6]:
def countsDF(source, conn):
    query = '''
    SELECT
      emperor,
      denomination,
      count(denomination) AS counts
    FROM
      {} JOIN emperors ON
        startDate >= emperors.start
        AND startDate <= emperors.end
    WHERE description LIKE '%radiate%'
    GROUP BY emperor, denomination
    ORDER BY start, end, counts DESC;
    '''.format(source)
    df = pd.read_sql(query, conn)
    return df

In [5]:
def ratiosDF(source, conn):
    query = '''
    WITH emperorCoins AS (
      SELECT
        emperor,
        description,
        denomination,
        start,
        end
      FROM
        {} JOIN emperors ON
          startDate >= emperors.start
          AND startDate <= emperors.end
    ), emperorTotals AS (
      SELECT
        emperor,
        count(description)  AS emperorCounts
      FROM emperorCoins
      GROUP BY emperor
    )
    SELECT
      emperor,
      count(description) * 1.0 / emperorCounts   AS keywordRatio
    FROM
      emperorCoins JOIN emperorTotals USING (emperor)
    WHERE description LIKE '%radiate%'
    GROUP BY emperor
    ORDER BY start, end;
    '''.format(source)
    df = pd.read_sql(query, conn)
    return df

## Generate Counts and Ratios Plots

In [3]:
def countsPlot(source, conn):
    df = countsDF(source, conn)
    counts = Bar(df, label=cat(columns="emperor", sort=False), palette=viridis(df.denomination.unique().size), 
                    values='counts', stack="denomination", responsive=True, active_scroll='wheel_zoom', 
                     title="Count of Radiate per Emperor in " + source + " Plot")

    hover_counts = HoverTool(tooltips=[
                                ("emperor", "@emperor"),
                                ("denomination", "@denomination"),
                                ("count", "@counts")
                            ])

    counts.add_tools(hover_counts)
    counts.xaxis.axis_label = "Emperors"
    counts.yaxis.axis_label = "Counts"

    save(counts, "../Plots/" + source + "_counts_plot.html")
    
    return counts

In [7]:
def ratiosPlot(source, conn):
    df = ratiosDF(source, conn)
    ratios = Bar(df, label=cat(columns="emperor", sort=False), values='keywordRatio', 
                 responsive=True, active_scroll='wheel_zoom', legend=False,
                 title="Ratio of Radiate by Emperor in " + source + " Plot")

    hover_counts = HoverTool(tooltips=[
                                ("emperor", "@emperor"),
                                ("Percentage", "@percentages")
                            ])

    ratios.add_tools(hover_counts)
    ratios.xaxis.axis_label = "Emperors"
    ratios.yaxis.axis_label = "Percentages"

    save(ratios, "../Plots/" + source + "_ratios_plot.html")
    
    return ratios

# Create Counts and Ratios Plots

In [8]:
sources = ['britishMuseum', 'americanNumismaticSociety', 'hoxneHoard']
for source in sources:
    counts = countsPlot(source, conn)
    #show(counts)
    ratios = ratiosPlot(source, conn)
    #show(ratios)

# Statistical Significance of Data

In [11]:
bm_ratio = ratiosDF('britishMuseum', conn)
ans_ratio = ratiosDF('americanNumismaticSociety', conn)
hoxne_ratio = ratiosDF('hoxneHoard', conn)
merged = bm_ratio.merge(ans_ratio, on='emperor')

In [12]:
merged.tail()

,emperor,keywordRatio_x,keywordRatio_y
49,Severus II,0.005988,0.008333
50,Maxentius,0.016529,0.011268
51,Constantine the Great,0.028852,0.037863
52,Licinius I,0.038914,0.067402
53,Maximinus II,0.013245,0.058824


In [20]:
stats.linregress(merged['keywordRatio_x'], merged['keywordRatio_y'])

LinregressResult(slope=0.51060994004261462, intercept=0.11313650245851507, rvalue=0.47854626950096807, pvalue=0.00025198293393652121, stderr=0.12992388768861643)

In [14]:
merged = merged.merge(hoxne_ratio, on='emperor')

In [16]:
merged.tail()

,emperor,keywordRatio_x,keywordRatio_y,keywordRatio
0,Gallienus,0.439153,0.546926,1.0
1,Claudius Gothicus,0.661157,0.496454,1.0


In [9]:
# Create counts of the coins by emperor in the hoxne hoard
query = '''
SELECT
  emperor,
  denomination,
  count(denomination) AS counts
FROM
  hoxneHoard JOIN emperors ON
    startDate >= emperors.start
    AND startDate <= emperors.end
GROUP BY emperor, denomination
ORDER BY start, end, counts DESC;
'''
df = pd.read_sql(query, conn)
counts = Bar(df, label=cat(columns="emperor", sort=False), palette=viridis(df.denomination.unique().size), 
                values='counts', stack="denomination", responsive=True, active_scroll='wheel_zoom', 
                 title="Count of coins by Emperor in Hoxne Hoard Plot")

hover_counts = HoverTool(tooltips=[
                            ("emperor", "@emperor"),
                            ("denomination", "@denomination"),
                            ("count", "@counts")
                        ])

counts.add_tools(hover_counts)
counts.xaxis.axis_label = "Emperors"
counts.yaxis.axis_label = "Counts"

save(counts, "../Plots/hoxneHoard_counts_plot.html")

'/Users/William/Documents/roman_coinage/Plots/hoxneHoard_counts_plot.html'